In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24273168
paper_name = 'ruggles_sturley_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/jbc.M113.515197-2.xlsx', sheet_name='Table S1A', skiprows=2)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 152 x 21


In [13]:
original_data.head()

,Unnamed: 0,Rank,Yeast Gene,ORF,Rate,time,OD,Rate.1,Time,OD.1,...,Calculated Ratio,p-value,Gene Function,GO PROCESS,Complex,Human Ortholog (Sequence Identity),human orthologous function,"Altered UPRE reporter levels (Jonikas, 2009)",Previous LD screens,YPO screens
0,NaN,1,ARV1,YLR242C,10.102732,31.0001,5.973771,0.1074,0.0012,0.0025,...,-1.969573,2.570400e-03,Protein functioning in transport of glycosylph...,LIPID METABOLISM,NaN,ARV1,"ARV1 ortholog, restores defective sterol uptak...",4.042453,"Increased (Fei, 2008)",NaN
1,NaN,2,VMA1,YDL185W,5.735593,11.9994,4.277698,0.0113,0.0001,0.0293,...,-4.787492,1.483480e-14,Subunit A of the eight-subunit V1 peripheral m...,"DNA METABOLISM, CELLULAR HOMEOSTASIS","H+-transporting ATPase, Vacuolar",ATP6V1A (37.8%),"ATPase H+ transporting lysosomal subunit A1, a...",NaN,"Decreased (Fei, 2008) Larger (Szymanski, 20...",NaN
2,NaN,3,IES6,YEL044W,5.927361,7.0017,3.598350,0.0217,0.0704,0.0439,...,-2.995732,2.393600e-06,Protein that associates with the INO80 chromat...,UNKNOWN,Ino80 complex,NaN,NaN,NaN,NaN,NaN
3,NaN,4,ARO1,YDR127W,5.080243,19.0002,3.461699,0.0602,0.0002,0.0708,...,-3.465169,3.893000e-08,"Pentafunctional arom protein, catalyzes steps ...",AMINO ACID METABOLISM,NaN,NaN,NaN,1.126595,NaN,NaN
4,NaN,5,VPS69,YPR087W,5.152941,8.0012,3.383636,0.1858,0.1048,0.0435,...,-1.837296,5.181600e-03,"Dubious open reading frame, unlikely to encode...",TRANSPORT PEROXISOME ORGANIZATION,NaN,NaN,NaN,NaN,"Increased (Fei, 2008)","OA Sensitive (Lockshon, 2007) OA Sensitive (Sm..."


In [14]:
original_data['orf'] = original_data['ORF'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Rank, Yeast Gene , ORF, Rate, time, OD, Rate.1, Time, OD.1, Toxicity score, Calculated Ratio , p-value , Gene Function , GO PROCESS, Complex, Human Ortholog (Sequence Identity), human orthologous function, Altered UPRE reporter levels (Jonikas, 2009), Previous LD screens, YPO screens , orf]
Index: []

[0 rows x 22 columns]


In [18]:
original_data['data'] = original_data['Calculated Ratio ']

In [19]:
original_data.set_index('orf', inplace=True)

In [20]:
original_data = original_data[['data']].copy()

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(152, 1)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [15989]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,15989
data_type,value
orf,
YAL013W,-0.992019
YBL002W,-1.977412
YBL006C,-3.179322
YBL021C,-1.958689
YBL094C,-2.053961


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,15989
,data_type,value
gene_id,orf,
11,YAL013W,-0.992019
90,YBL002W,-1.977412
94,YBL006C,-3.179322
109,YBL021C,-1.958689
182,YBL094C,-2.053961


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          15989          
data_type           value    valuez
gene_id orf                        
11      YAL013W -0.992019 -2.096690
90      YBL002W -1.977412 -4.127962
94      YBL006C -3.179322 -6.605558
109     YBL021C -1.958689 -4.089366
182     YBL094C -2.053961 -4.285760

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24273168...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.41it/s]

Updating the data_modified_on field...
